In [ ]:
import gzip
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import LSTM, GRU, Dense, RepeatVector, TimeDistributed, Input, \
    multiply, concatenate, Flatten, Activation, Bidirectional, Embedding, Dropout, Conv1D, \
    MultiHeadAttention, Add, LayerNormalization, BatchNormalization, Dot


from keras.models import Sequential, Model, load_model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model, model_to_dot

from tensorflow.keras.regularizers import L1, L2


In [ ]:
PSSP_model = tf.keras.models.load_model('model.h5')

In [ ]:
#import data CB513

import gzip
import numpy as np

f = gzip.GzipFile('Data/cb513_profile_MASK.npy.gz', "r")
cb513_profile= np.load(f)
f.close()

f = gzip.GzipFile('Data/cb513_sequence_MASK.npy.gz', "r")
cb513_sequence= np.load(f)
f.close()


cb513_sequence_integer = np.zeros((cb513_sequence.shape[0], cb513_sequence.shape[1]))
print(cb513_sequence_integer.shape)

for i in range(0, cb513_sequence_integer.shape[0]):
    for j in range(0, cb513_sequence_integer.shape[1]):
        cb513_sequence_integer[i][j] = np.argmax(cb513_sequence[i][j])

print(cb513_sequence_integer)

cb513_hmm_profile = np.load('Data/hhm_test.npy', "r")
print(cb513_hmm_profile.shape)
print(cb513_hmm_profile)

f = gzip.GzipFile('Data/cb513_testlabel_MASK.npy.gz', "r")
cb513_testlabel = np.load(f)
f.close()

In [ ]:
#print(cb513_testdata.shape)
print(cb513_testlabel.shape)

In [ ]:
loss, acc = PSSP_model.evaluate([cb513_sequence_integer, cb513_profile, cb513_hmm_profile], cb513_testlabel, verbose=2)


print('Restored PSSP model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
prediction_result = PSSP_model.predict([cb513_sequence_integer, cb513_profile, cb513_hmm_profile])



In [ ]:
print(prediction_result)

In [ ]:
print(prediction_result.shape)

In [ ]:
class_prediction_result = np.zeros((514,700,9))
class_prediction_result.shape

In [ ]:


for i in range(0, len(prediction_result)):
    for j in range(0, len(prediction_result[i])):
        index = np.argmax(prediction_result[i][j])
        class_prediction_result[i][j][index]= 1

In [ ]:

cb513_testlabel_reshape=cb513_testlabel.reshape(-1,9)
print(cb513_testlabel_reshape.shape)

class_prediction_result_reshape = class_prediction_result.reshape(-1,9)
print(class_prediction_result_reshape.shape)

In [ ]:
#transform class label into a single digit (integer 0..9)
cb513_testlabel_value = np.zeros((cb513_testlabel_reshape.shape[0],1))

for i in range(0, len(cb513_testlabel_reshape)):
    value = np.argmax(cb513_testlabel_reshape[i])
    cb513_testlabel_value[i] = value

In [ ]:
class_prediction_result_value = np.zeros((class_prediction_result_reshape.shape[0],1))

for i in range(0, len(class_prediction_result_reshape)): 
    value = np.argmax(class_prediction_result_reshape[i])
    class_prediction_result_value[i] = value


In [ ]:
#'L', 'B', 'E', 'G', 'I', 'H', 'S', 'T','NoSeq'

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(cb513_testlabel_value,class_prediction_result_value))
print(classification_report(cb513_testlabel_value,class_prediction_result_value, labels=[0,1,2,3,4,5,6,7,8]))